In [2]:
import os
import re

import pandas as pd
import tkinter as tk

from datetime import datetime
from smartapi import SmartConnect
from symbol_selector import UserInterface


In [3]:
uname = "SBD1199"
pwd = "Pass2406@ab"
api_key = "CGz5s9Tv"
api_key_feed = "bgBHbiTo"
secret = "1c873097-b4f6-4b50-ae5f-5b70341a0347"
tokens_url = "https://margincalculator.angelbroking.com/OpenAPI_File/files/OpenAPIScripMaster.json"

df_master = pd.read_json(tokens_url)
s_map = pd.Series(df_master.name.values,index=df_master.symbol).to_dict()
s_concat_list = ["{}  ||  {}".format(key, value) for key, value in s_map.items()]


In [4]:
obj = SmartConnect(api_key=api_key)
data = obj.generateSession(uname, pwd)
refresh_token = data['data']['refreshToken']
feedtoken = obj.getfeedToken()
user_profile = obj.getProfile(refresh_token)

In [5]:
def get_historic_data(exchange, symbol, token, interval, fdate, todate):
    try:
        params={
            "exchange": exchange,
            "tradingsymbol": symbol,
            "symboltoken": token,
            "interval": interval,
            "fromdate": fdate,
            "todate": todate
        }
        candle_data = obj.getCandleData(params)['data']
        df_candle = pd.DataFrame(candle_data)
        
        df_candle = df_candle.rename(
                    columns = {0: "Datetime", 1: "Open", 2: "High", 3: "Low", 4: "Close", 5: "Volume"}
                   )
        df_candle['Datetime'] = pd.to_datetime(df_candle['Datetime'])
        df_candle = df_candle.set_index('Datetime')
        return df_candle
    except Exception as e:
        print("Something went wrong - {}".format(e))
        return None
        

In [6]:
def serialize_symbol(symbol):
    index = str(re.findall('([a-zA-Z ]*)\d*.*', symbol)[0]).upper()
    symbol = symbol.upper().lstrip(index)
    date_pattern = '^(\d{2})\w{3}\d{2}'
    expiry = (re.match(date_pattern, symbol)[0])
    strike_price = symbol[7:len(symbol)-2]
    return(str(index), str(expiry), int(strike_price))


def get_expiry_data(symbol, increment, limit):
    index, expiry, strike_price = serialize_symbol(symbol)
    option_list = ['CE', 'PE']
    l_range = int((limit-strike_price)/50)
    l_range = l_range+1
    derived_symbols = list()
    for i in range(0, l_range):
        for j in option_list:
            end_str = str(strike_price + (i*increment)) + j
            derived_symbols.append(index + expiry + end_str)
    return derived_symbols


In [8]:
import time
symbol = 'BANKNIFTY16SEP2130400CE'
increment= 100
limit = 40900
derived_symbols = get_expiry_data(symbol=symbol, increment=increment, limit=limit)

exchange = 'NFO'
timeframe = "ONE_MINUTE"
from_date = "2021-08-01 00:00"
to_date = "2021-09-17 23:59"


# parent_dir = "C:\\Users\\haris\\Desktop\\Autodha"
parent_dir = os.getcwd()
directory = symbol
path = os.path.join(parent_dir, directory)
c_path = os.path.join(path, 'CE')
p_path = os.path.join(path, 'PE')

os.makedirs(path, exist_ok=True)
os.makedirs(c_path, exist_ok=True)
os.makedirs(p_path, exist_ok=True)

run_count = 0
pass_count = 0

for enum, each_symbol in enumerate(derived_symbols):
    time.sleep(1)
    print(enum)
    run_count+=1
    ser = df_master.loc[df_master['symbol'] == each_symbol]
    each_token = str(ser['token'].to_list()[0])
    print(each_symbol, each_token)
    daily_data = get_historic_data(exchange, each_symbol, each_token, timeframe, from_date, to_date)
    result_df = pd.DataFrame(daily_data)
    curr_time = datetime.now()
    ctime = curr_time.strftime("%Y-%m-%d_%H-%M-%S")] 
    if not result_df.empty: 
        if each_symbol.endswith('CE'):
            result_df.to_csv("{}\\{}_{}.csv".format(c_path, each_symbol, ctime), index=True)

        if each_symbol.endswith('CE'):
            result_df.to_csv("{}\\{}_{}.csv".format(p_path, each_symbol, ctime), index=True)

        pass_count+=1
        print("csv file has been created for {} at {}".format(each_symbol, ctime))

        

0
BANKNIFTY16SEP2130400CE 44131
csv file has been created for BANKNIFTY16SEP2130400CE at 2021-09-17_00-28-36
1
BANKNIFTY16SEP2130400PE 44132
csv file has been created for BANKNIFTY16SEP2130400PE at 2021-09-17_00-28-38
2
BANKNIFTY16SEP2130500CE 44139
csv file has been created for BANKNIFTY16SEP2130500CE at 2021-09-17_00-28-39
3
BANKNIFTY16SEP2130500PE 44142
csv file has been created for BANKNIFTY16SEP2130500PE at 2021-09-17_00-28-40
4
BANKNIFTY16SEP2130600CE 44143
Something went wrong - 'Datetime'
5
BANKNIFTY16SEP2130600PE 44144
csv file has been created for BANKNIFTY16SEP2130600PE at 2021-09-17_00-28-43
6
BANKNIFTY16SEP2130700CE 44145
Something went wrong - 'Datetime'
7
BANKNIFTY16SEP2130700PE 44146
csv file has been created for BANKNIFTY16SEP2130700PE at 2021-09-17_00-28-45
8
BANKNIFTY16SEP2130800CE 44152
Something went wrong - 'Datetime'
9
BANKNIFTY16SEP2130800PE 44153
csv file has been created for BANKNIFTY16SEP2130800PE at 2021-09-17_00-28-48
10
BANKNIFTY16SEP2130900CE 44154
Someth

84
BANKNIFTY16SEP2134600CE 45961
csv file has been created for BANKNIFTY16SEP2134600CE at 2021-09-17_00-30-23
85
BANKNIFTY16SEP2134600PE 45962
csv file has been created for BANKNIFTY16SEP2134600PE at 2021-09-17_00-30-25
86
BANKNIFTY16SEP2134700CE 45963
csv file has been created for BANKNIFTY16SEP2134700CE at 2021-09-17_00-30-26
87
BANKNIFTY16SEP2134700PE 45964
csv file has been created for BANKNIFTY16SEP2134700PE at 2021-09-17_00-30-28
88
BANKNIFTY16SEP2134800CE 45965
csv file has been created for BANKNIFTY16SEP2134800CE at 2021-09-17_00-30-29
89
BANKNIFTY16SEP2134800PE 45966
csv file has been created for BANKNIFTY16SEP2134800PE at 2021-09-17_00-30-31
90
BANKNIFTY16SEP2134900CE 45971
csv file has been created for BANKNIFTY16SEP2134900CE at 2021-09-17_00-30-32
91
BANKNIFTY16SEP2134900PE 45972
csv file has been created for BANKNIFTY16SEP2134900PE at 2021-09-17_00-30-33
92
BANKNIFTY16SEP2135000CE 45973
csv file has been created for BANKNIFTY16SEP2135000CE at 2021-09-17_00-30-35
93
BANKNIF

158
BANKNIFTY16SEP2138300CE 46109
csv file has been created for BANKNIFTY16SEP2138300CE at 2021-09-17_00-32-20
159
BANKNIFTY16SEP2138300PE 46111
csv file has been created for BANKNIFTY16SEP2138300PE at 2021-09-17_00-32-22
160
BANKNIFTY16SEP2138400CE 46120
csv file has been created for BANKNIFTY16SEP2138400CE at 2021-09-17_00-32-23
161
BANKNIFTY16SEP2138400PE 46124
csv file has been created for BANKNIFTY16SEP2138400PE at 2021-09-17_00-32-24
162
BANKNIFTY16SEP2138500CE 46125
csv file has been created for BANKNIFTY16SEP2138500CE at 2021-09-17_00-32-26
163
BANKNIFTY16SEP2138500PE 46127
csv file has been created for BANKNIFTY16SEP2138500PE at 2021-09-17_00-32-27
164
BANKNIFTY16SEP2138600CE 46129
csv file has been created for BANKNIFTY16SEP2138600CE at 2021-09-17_00-32-29
165
BANKNIFTY16SEP2138600PE 46131
csv file has been created for BANKNIFTY16SEP2138600PE at 2021-09-17_00-32-30
166
BANKNIFTY16SEP2138700CE 46134
csv file has been created for BANKNIFTY16SEP2138700CE at 2021-09-17_00-32-32
1

IndexError: list index out of range